In [1]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs

In [8]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%97%AD%EB%8C%80+%EB%B0%95%EC%8A%A4%EC%98%A4%ED%94%BC%EC%8A%A4%EC%88%9C%EC%9C%84'

res = req.get(url)
soup = bs(res.text, 'html.parser')
movierank_li = soup.select('._content > ul > li')

In [77]:
movie_title = []
movie_date = []
movie_ticketsales = []
movie_url = []

In [28]:
# 제목
movierank_li[0].select('.movie_tit strong')[0].text

'명량'

In [32]:
# 개봉일
movierank_li[0].select('.movie_item dd')[0].text

' 2014.07.30. '

In [33]:
# 관객수
movierank_li[0].select('.movie_item dd')[1].text

'17,615,844명'

In [48]:
movierank_li[0].select('.btn_area > a')[0].attrs['href']

'https://serieson.naver.com/tvstore/detail.nhn?mcode=93756'

In [54]:
movierank_li[0].select('.btn_area > a')[0].text

'다운로드'

In [78]:
for movie in movierank_li:
    movie_title.append(movie.select('.movie_tit strong')[0].text)
    movie_date.append(movie.select('.movie_item dd')[0].text)
    movie_ticketsales.append(movie.select('.movie_item dd')[1].text)
    
    tmp = movie.select('.btn_area > a')
    
    if len(tmp) == 2:
        if tmp[0].text == '다운로드':
            movie_url.append(tmp[0].attrs['href'])
        else:
            movie_url.append(tmp[1].attrs['href'])
    else:
        if tmp[0].text == '예고편':
            movie_url.append(tmp[0].attrs['href'].split('&')[0])
        else:
            movie_url.append(tmp[0].attrs['href'])

movie_dict = {
    'movie_title':movie_title,
    'movie_date':movie_date,
    'movie_ticketsales':movie_ticketsales,
    'movie_url' : movie_url
}

In [79]:
tmp_df = pd.DataFrame(movie_dict)
tmp_df.head()

,movie_title,movie_date,movie_ticketsales,movie_url
0,명량,2014.07.30.,"17,615,844명",https://serieson.naver.com/tvstore/detail.nhn?...
1,극한직업,2019.01.23.,"16,266,338명",https://serieson.naver.com/tvstore/detail.nhn?...
2,신과함께-죄와 벌,2017.12.20.,"14,414,658명",https://serieson.naver.com/tvstore/detail.nhn?...
3,국제시장,2014.12.17.,"14,264,059명",https://serieson.naver.com/tvstore/detail.nhn?...
4,어벤져스: 엔드게임,2019.04.24.,"13,977,602명",https://serieson.naver.com/tvstore/detail.nhn?...


In [63]:
tmp_df['movie_url'][0].split('?')[1]

'mcode=93756'

In [80]:
tmp_df['movie_url'] = tmp_df['movie_url'].apply(lambda x: x.split('?')[1])

In [81]:
tmp_df['movie_url'][41]

'code=39569'

In [82]:
tmp_df['movie_code'] = tmp_df['movie_url'].apply(lambda x: x.split('=')[1])

In [83]:
tmp_df['movie_code']

0      93756
1     167651
2      85579
3     102875
4     136900
       ...  
95    163533
96     52548
97    137326
98     49727
99    144314
Name: movie_code, Length: 100, dtype: object

In [84]:
code_list = tmp_df['movie_code']

target_url = 'https://movie.naver.com/movie/bi/mi/point.naver?'


res = req.get(target_url, params={'code':code_list[0]})
res


<Response [200]>

In [142]:
soup = bs(res.text, 'html.parser')
tmp_spc_score = soup.select('.spc_score_area')[0].select('div > em')
spc_score = ''.join([i.text for i in tmp_spc_score])
spc_score

'6.29'

In [165]:
tmp_ntz_score = soup.select('.score_left > .star_score em')
ntz_score = ''.join(i.text for i in tmp_ntz_score)
ntz_score

'8.44'

In [202]:
# page5까지
target_url2 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?'

param_dict = {
    'code':'93756',
    'type':'after',
    'onlyActualPointYn':'Y',
    'onlySpoilerPointYn':'N',
    'order':'sympathyScore',
    'page':1
}

res2 = req.get(target_url2, params=param_dict)
soup2 = bs(res2.text, 'html.parser')

In [209]:
tmp_act_score = soup2.select('.score_result .star_score em')
act_score = list(map(int, [i.text for i in tmp_act_score]))
sum(act_score)

97

In [210]:
# 50개 평균
target_url2 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?'
avg_act_score = 0

for i in range(1,5):
    
    param_dict = {
        'code':'93756',
        'type':'after',
        'onlyActualPointYn':'Y',
        'onlySpoilerPointYn':'N',
        'order':'sympathyScore',
        'page':i
    }

    res2 = req.get(target_url2, params=param_dict)
    soup2 = bs(res2.text, 'html.parser')
    tmp_act_score = soup2.select('.score_result .star_score em')
    act_score = list(map(int, [i.text for i in tmp_act_score]))
    
    avg_act_score += sum(act_score)
    
avg_act_score /= 50
avg_act_score

7.14

In [216]:
# 전체 평점 긁기
target_url = 'https://movie.naver.com/movie/bi/mi/point.naver?'
target_url2 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?'

actual_valence = []
netizen_valence = []
critic_valence = []

for movie_code in code_list:
    
    res = req.get(target_url, params={'code':movie_code})
    soup = bs(res.text, 'html.parser')
    
    # 전문가 평점
    try:
        tmp_spc_score = soup.select('.spc_score_area')[0].select('div > em')
        spc_score = ''.join([i.text for i in tmp_spc_score])
        critic_valence.append(float(spc_score))
    except:
        critic_valence.append(0)
    
    # 네티즌 평점
    try:
        tmp_ntz_score = soup.select('.score_left > .star_score em')
        ntz_score = ''.join(i.text for i in tmp_ntz_score)
        netizen_valence.append(float(ntz_score))
    except:
        netizen_valence.append(0)
    
    # 관람객 공감순 50개 평균 평점
    avg_act_score = 0

    for j in range(1,5):

        param_dict = {
            'code':movie_code,
            'type':'after',
            'onlyActualPointYn':'Y',
            'onlySpoilerPointYn':'N',
            'order':'sympathyScore',
            'page':j
        }

        res2 = req.get(target_url2, params=param_dict)
        soup2 = bs(res2.text, 'html.parser')
        tmp_act_score = soup2.select('.score_result .star_score em')
        act_score = list(map(int, [i.text for i in tmp_act_score]))

        avg_act_score += sum(act_score)

    avg_act_score /= 50
    actual_valence.append(avg_act_score)

In [218]:
movie_dict

{'movie_title': ['명량',
  '극한직업',
  '신과함께-죄와 벌',
  '국제시장',
  '어벤져스: 엔드게임',
  '겨울왕국 2',
  '베테랑',
  '아바타',
  '도둑들',
  '7번방의 선물',
  '알라딘',
  '암살',
  '범죄도시2',
  '광해, 왕이 된 남자',
  '신과함께-인과 연',
  '택시운전사',
  '부산행',
  '변호인',
  '해운대',
  '어벤져스: 인피니티 워',
  '실미도',
  '괴물',
  '왕의 남자',
  '어벤져스: 에이지 오브 울트론',
  '인터스텔라',
  '기생충',
  '겨울왕국',
  '보헤미안 랩소디',
  '검사외전',
  '엑시트',
  '설국열차',
  '관상',
  '아이언맨 3',
  '캡틴 아메리카: 시빌 워',
  '해적: 바다로 간 산적',
  '수상한 그녀',
  '국가대표',
  '백두산',
  '과속스캔들',
  '탑건: 매버릭',
  '스파이더맨: 파 프롬 홈',
  '디 워',
  '공조',
  '트랜스포머 3',
  '히말라야',
  '스파이더맨: 노 웨이 홈',
  '미션 임파서블 : 고스트 프로토콜',
  '밀정',
  '최종병기 활',
  '써니',
  '트랜스포머',
  '트랜스포머: 패자의 역습',
  '스파이더맨: 홈커밍',
  '한산: 용의 출현',
  '1987',
  '베를린',
  '마스터',
  '터널',
  '어벤져스',
  '내부자들',
  '늑대소년',
  '인천상륙작전',
  '럭키',
  '은밀하게 위대하게',
  '범죄도시',
  '곡성(哭聲)',
  '화려한 휴가',
  '좋은 놈, 나쁜 놈, 이상한 놈',
  '공조2: 인터내셔날',
  '군함도',
  '미션 임파서블: 폴아웃',
  '웰컴 투 동막골',
  '다크 나이트 라이즈',
  '사도',
  '아저씨',
  '킹스맨 : 시크릿 에이전트',
  '미션 임파서블: 로그네이션',
  '미녀는 괴로워',
  '전우치',
  '연평해전',
  '인셉션',
  '

In [219]:
movie_dict['critic_valence'] = critic_valence
movie_dict['netizen_valence'] = netizen_valence
movie_dict['actual_valence'] = actual_valence

In [220]:
df = pd.DataFrame(movie_dict)
df.head()

,movie_title,movie_date,movie_ticketsales,movie_url,critic_valence,netizen_valence,actual_valence
0,명량,2014.07.30.,"17,615,844명",https://serieson.naver.com/tvstore/detail.nhn?...,6.29,8.44,7.14
1,극한직업,2019.01.23.,"16,266,338명",https://serieson.naver.com/tvstore/detail.nhn?...,6.80,8.51,7.30
2,신과함께-죄와 벌,2017.12.20.,"14,414,658명",https://serieson.naver.com/tvstore/detail.nhn?...,5.92,7.83,4.28
3,국제시장,2014.12.17.,"14,264,059명",https://serieson.naver.com/tvstore/detail.nhn?...,5.81,9.02,7.72
4,어벤져스: 엔드게임,2019.04.24.,"13,977,602명",https://serieson.naver.com/tvstore/detail.nhn?...,7.62,9.38,7.92


In [222]:
df['critic_actual_diff'] = abs(df['actual_valence'] - df['critic_valence'])

In [225]:
df = df.sort_values('critic_actual_diff')
df.reset_index(inplace=True, drop=True)
df.loc[(df['critic_valence'] != 0) & (df['actual_valence'] != 0) & (df['critic_actual_diff']==0)]

,index,movie_title,movie_date,movie_ticketsales,movie_url,critic_valence,netizen_valence,actual_valence,critic_netizen_diff,critic_actual_diff
3,76,미션 임파서블: 로그네이션,2015.07.30.,"6,126,488명",https://serieson.naver.com/tvstore/detail.nhn?...,7.54,8.75,7.54,0.0,0.0
